# Iris Species Classification

This notebook trains a Random Forest classifier to predict iris species.

**Dataset**: Iris Dataset (Fisher, 1936)
- 150 samples of iris flowers
- Target: Species (Setosa, Versicolor, Virginica)

**Features**:
- `sepal_length`: Sepal length (cm)
- `sepal_width`: Sepal width (cm)
- `petal_length`: Petal length (cm)
- `petal_width`: Petal width (cm)


In [ ]:
%pip install pandas numpy scikit-learn mlflow


In [ ]:
import os
import sys
import argparse
import json
import tempfile
import numpy as np
import pandas as pd
from datetime import datetime

# MLflow imports
import mlflow
import mlflow.sklearn
from mlflow import set_tracking_uri, set_experiment
from mlflow.client import MlflowClient
from mlflow.models import infer_signature

# Scikit-learn imports
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
def setup_mlflow(mlflow_uri: str, username: str, password: str) -> MlflowClient:
    """Configure MLflow tracking and return client."""
    os.environ["MLFLOW_TRACKING_USERNAME"] = username
    os.environ["MLFLOW_TRACKING_PASSWORD"] = password
    
    set_tracking_uri(mlflow_uri)
    client = MlflowClient(mlflow_uri)
    
    print(f"✅ MLflow tracking URI: {mlflow_uri}")
    return client


def load_and_prepare_data():
    """Load Iris dataset and prepare train/test splits."""
    print("\n" + "=" * 80)
    print("📊 LOADING DATASET")
    print("=" * 80)
    
    iris = load_iris(as_frame=True)
    X = iris.data
    y = iris.target
    
    # Rename columns to match feature names without spaces
    X.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    
    # Target names
    target_names = iris.target_names.tolist()
    
    print(f"Dataset: Iris")
    print(f"Samples: {X.shape[0]:,}")
    print(f"Features: {X.shape[1]}")
    print(f"\nFeature names:")
    for i, col in enumerate(X.columns, 1):
        print(f"  {i}. {col}")
    
    print(f"\nTarget classes:")
    for i, name in enumerate(target_names):
        count = (y == i).sum()
        print(f"  {i}. {name}: {count} samples")
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
    )
    
    print(f"\nTrain samples: {X_train.shape[0]:,}")
    print(f"Test samples: {X_test.shape[0]:,}")
    
    return X_train, X_test, y_train, y_test, X.columns.tolist(), target_names


def train_model(X_train, y_train, X_test, y_test, hyperparams: dict):
    """Train Random Forest classifier and return predictions."""
    print("\n" + "=" * 80)
    print("🚀 TRAINING MODEL")
    print("=" * 80)
    
    print("Hyperparameters:")
    for key, value in hyperparams.items():
        print(f"  {key}: {value}")
    
    model = RandomForestClassifier(**hyperparams)
    model.fit(X_train, y_train)
    
    print("✅ Training completed!")
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    return model, y_train_pred, y_test_pred


def calculate_metrics(y_true, y_pred, dataset_name="Test"):
    """Calculate and return evaluation metrics."""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    return {
        f"{dataset_name.lower()}_accuracy": accuracy,
        f"{dataset_name.lower()}_precision": precision,
        f"{dataset_name.lower()}_recall": recall,
        f"{dataset_name.lower()}_f1": f1
    }


def log_to_mlflow(model, X_train, y_train, X_test, y_test, 
                  y_train_pred, y_test_pred, hyperparams, feature_names, target_names):
    """Log model, parameters, and metrics to MLflow."""
    print("\n" + "=" * 80)
    print("📝 LOGGING TO MLFLOW")
    print("=" * 80)
    
    # Log hyperparameters
    for key, value in hyperparams.items():
        mlflow.log_param(key, value)
    
    # Calculate and log metrics
    train_metrics = calculate_metrics(y_train, y_train_pred, "Train")
    test_metrics = calculate_metrics(y_test, y_test_pred, "Test")
    all_metrics = {**train_metrics, **test_metrics}
    
    for metric_name, metric_value in all_metrics.items():
        mlflow.log_metric(metric_name, metric_value)
    
    print("\n📈 Model Performance:")
    print(f"  Training Accuracy: {train_metrics['train_accuracy']:.4f}")
    print(f"  Training F1: {train_metrics['train_f1']:.4f}")
    print(f"  Test Accuracy: {test_metrics['test_accuracy']:.4f}")
    print(f"  Test Precision: {test_metrics['test_precision']:.4f}")
    print(f"  Test Recall: {test_metrics['test_recall']:.4f}")
    print(f"  Test F1: {test_metrics['test_f1']:.4f}")
    
    # Print confusion matrix
    cm = confusion_matrix(y_test, y_test_pred)
    print(f"\n  Confusion Matrix:")
    print(f"  {cm}")
    
    # Create model signature
    signature = infer_signature(X_train, model.predict(X_train))
    input_example = X_test.head(1)
    
    # Save and log model
    with tempfile.TemporaryDirectory() as tmpdir:
        local_model_path = os.path.join(tmpdir, "model")
        
        mlflow.sklearn.save_model(
            model,
            local_model_path,
            signature=signature,
            input_example=input_example
        )
        
        mlflow.log_artifacts(local_model_path, artifact_path="model")
        print("✅ Model artifacts logged successfully!")
    
    return all_metrics


def create_sample_payload(X_test, y_test, model, feature_names, target_names):
    """Create realistic sample prediction payload."""
    # Get one sample from each class if possible
    samples = []
    
    for class_idx in range(len(target_names)):
        class_samples = X_test[y_test == class_idx]
        if len(class_samples) > 0:
            sample = class_samples.iloc[0]
            actual_class = class_idx
            predicted_class = model.predict(sample.values.reshape(1, -1))[0]
            
            samples.append({
                "features": sample.to_dict(),
                "actual_class": int(actual_class),
                "actual_species": target_names[actual_class],
                "predicted_class": int(predicted_class),
                "predicted_species": target_names[predicted_class]
            })
    
    # Return first sample for main payload
    return samples[0] if samples else None


def register_model(client: MlflowClient, model_name: str, run_id: str, experiment_id: str):
    """Register model in MLflow Model Registry."""
    print("\n" + "=" * 80)
    print("📦 REGISTERING MODEL")
    print("=" * 80)
    
    model_uri = f"runs:/{run_id}/model"
    
    # Create registered model if it doesn't exist
    try:
        client.get_registered_model(model_name)
        print(f"ℹ️  Model '{model_name}' already exists in registry")
    except Exception:
        try:
            client.create_registered_model(model_name)
            print(f"✅ Created registered model: {model_name}")
        except Exception as e:
            print(f"⚠️  Could not create registered model: {e}")
    
    # Create model version
    try:
        result = client.create_model_version(
            name=model_name,
            source=model_uri,
            run_id=run_id
        )
        print(f"✅ Model version registered successfully!")
        print(f"   Model Name: {model_name}")
        print(f"   Version: {result.version}")
        print(f"   Run ID: {run_id}")
        return result.version
    except Exception as e:
        print(f"⚠️  Model registration failed (model still usable via run URI): {e}")
        print(f"   You can deploy using: mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model")
        return None


def print_deployment_info(run_id: str, experiment_id: str, sample_payload: dict):
    """Print deployment instructions and sample payloads."""
    print("\n" + "=" * 80)
    print("🎉 TRAINING COMPLETE!")
    print("=" * 80)
    
    print(f"\n📋 Run Information:")
    print(f"  Run ID: {run_id}")
    print(f"  Experiment ID: {experiment_id}")
    print(f"  Model URI: mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model")
    
    print("\n" + "=" * 80)
    print("🚀 DEPLOYMENT PAYLOAD (deploy-model API)")
    print("=" * 80)
    
    deploy_payload = {
        "serve_name": "iris-classifier",
        "model_uri": f"mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model",
        "env": "local",
        "cores": 2,
        "memory": 4,
        "node_capacity": "spot",
        "min_replicas": 1,
        "max_replicas": 3
    }
    
    print(json.dumps(deploy_payload, indent=2))
    
    print("\n" + "=" * 80)
    print("📊 SAMPLE PREDICTION PAYLOAD")
    print("=" * 80)
    
    predict_payload = {
        "features": sample_payload["features"]
    }
    
    print(json.dumps(predict_payload, indent=2))
    
    print(f"\n💡 Expected Output:")
    print(f"  Actual Species: {sample_payload['actual_species']} (class {sample_payload['actual_class']})")
    print(f"  Model Prediction: {sample_payload['predicted_species']} (class {sample_payload['predicted_class']})")
    
    print("\n" + "=" * 80)
    print("📝 FEATURE DESCRIPTIONS")
    print("=" * 80)
    features = sample_payload["features"]
    print(f"""
  Sepal Length: {features['sepal_length']:.2f} cm
  Sepal Width:  {features['sepal_width']:.2f} cm
  Petal Length: {features['petal_length']:.2f} cm
  Petal Width:  {features['petal_width']:.2f} cm
    """)


In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Train Iris Classification Model")
    parser.add_argument(
        "--mlflow-uri",
        default="http://darwin-mlflow-lib.darwin.svc.cluster.local:8080",
        help="MLflow tracking URI"
    )
    parser.add_argument(
        "--username",
        default="dhruv.mehta@dream11.com",
        help="MLflow username"
    )
    parser.add_argument(
        "--password",
        default="dhruv.mehta@dream11.com",
        help="MLflow password"
    )
    parser.add_argument(
        "--experiment-name",
        default="iris_classification",
        help="MLflow experiment name"
    )
    parser.add_argument(
        "--model-name",
        default="IrisClassifierRF",
        help="Registered model name"
    )
    
    args, _ = parser.parse_known_args()
    
    print("\n" + "=" * 80)
    print("🌸 IRIS SPECIES CLASSIFICATION MODEL TRAINING")
    print("=" * 80)
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Setup MLflow
    client = setup_mlflow(args.mlflow_uri, args.username, args.password)
    set_experiment(experiment_name=args.experiment_name)
    print(f"✅ Experiment: {args.experiment_name}")
    
    # Load data
    X_train, X_test, y_train, y_test, feature_names, target_names = load_and_prepare_data()
    
    # Define hyperparameters
    hyperparams = {
        "n_estimators": 100,
        "max_depth": 10,
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "random_state": 42,
        "n_jobs": -1
    }
    
    # Start MLflow run
    with mlflow.start_run(run_name=f"rf_iris_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
        # Train model
        model, y_train_pred, y_test_pred = train_model(
            X_train, y_train, X_test, y_test, hyperparams
        )
        
        # Log to MLflow
        metrics = log_to_mlflow(
            model, X_train, y_train, X_test, y_test,
            y_train_pred, y_test_pred, hyperparams, feature_names, target_names
        )
        
        # Get run information
        run_id = mlflow.active_run().info.run_id
        experiment_id = mlflow.active_run().info.experiment_id
        
        # Create sample payload
        sample_payload = create_sample_payload(X_test, y_test, model, feature_names, target_names)
    
    # Register model (outside of run context)
    version = register_model(client, args.model_name, run_id, experiment_id)
    
    # Print deployment information
    print_deployment_info(run_id, experiment_id, sample_payload)
    
    print("\n✅ Script completed successfully!")
    print("=" * 80 + "\n")


if __name__ == "__main__":
    main()
